In [1]:
import requests
import re
from bs4 import BeautifulSoup
import time

from lxml import etree
import pandas as pd


In [2]:
url = "https://www.maoyan.com/films/1299372"
url2 = "https://www.maoyan.com/ajax/films/1299372?timeStamp="+str(int(time.time()*1000) -3 )+"&index=2&signKey=893017c3979511c128ffc05dd2aca596&channelId=40011&sVersion=1&webdriver=false"

header = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}


    # time.sleep(1000)
# resp = requests.get(url = url2 , headers= header)

# print(len(resp.text))

In [6]:
url="https://www.maoyan.com/films/celebrity/ajax/works/2892300?offset=3"
for i in range(100):
    resp = requests.get("https://www.maoyan.com/films/celebrity/ajax/works/2892300?offset=3" , headers=header)
    print(len(resp.text))

14659


In [ ]:
import pandas as pd

url3 = "https://www.imdb.com/title/tt7362036/companycredits"

resp3 = requests.get(url=url3 , headers=header)

print(resp3.text)

In [75]:
url = "https://www.imdb.com/title/" + "tt7362036"  # 拼接得到URL
header = {
    "accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36",
    "sec-ch-us" : '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"'
}
resp = requests.get(url = url , headers = header)
resp.raise_for_status()
resp.encoding = resp.apparent_encoding


In [4]:
soup = BeautifulSoup(resp.text , "html.parser")
parse = etree.HTML(resp.text)

In [5]:
soup.find("section" , attrs = {"data-testid":"Storyline"})

In [232]:
# release date
soup.find("section" , attrs = {"data-testid":"Details"}).find("li" , attrs = {"data-testid":"title-details-releasedate"}).find('div').text

'July 5, 2018 (China)'

In [230]:
# Box office
soup.find("section" , attrs = {"data-testid":"BoxOffice"}).find("li" , attrs = {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).find('div').text

'$451,183,391'

In [19]:
soup.find("div" , attrs = {"data-testid":"genres"}).find_all("a")#[0].text

[<a aria-disabled="false" class="sc-16ede01-3 bYNgQ ipc-chip ipc-chip--on-baseAlt" href="/search/title?genres=comedy&amp;explore=title_type,genres&amp;ref_=tt_ov_inf" role="button" tabindex="0"><span class="ipc-chip__text">Comedy</span></a>,
 <a aria-disabled="false" class="sc-16ede01-3 bYNgQ ipc-chip ipc-chip--on-baseAlt" href="/search/title?genres=drama&amp;explore=title_type,genres&amp;ref_=tt_ov_inf" role="button" tabindex="0"><span class="ipc-chip__text">Drama</span></a>]

In [20]:
soup.find("h1" , attrs={"data-testid":"hero-title-block__title"}).text

'Wo bu shi yao shen'

In [ ]:
soup_full = None

In [97]:
full_list = soup_full.find("div" , attrs = {'id':'fullcredits_content'}).find_all("h4")

In [115]:
for i in range(len(full_list)):
    print(re.sub("\(.*\)" , "" , full_list[i].text).strip())

Directed by
Writing Credits
Cast
Produced by
Music by
Cinematography by
Film Editing by
Casting By
Production Design by
Costume Design by
Production Management
Second Unit Director or Assistant Director
Sound Department
Visual Effects by
Editorial Department
Location Management
Additional Crew
